### A hospital in the province of Greenland has been trying to improve its care conditions by looking at historic survival of the patients. They tried looking at their data but could not identify the main factors leading to high survivals.

####   Developing a model that will predict the chances of survival of a patient after 1 year of treatment (Survived_1_year).


 Dataset :https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Training_set_begs.csv
        

##### Testing
Dataset:https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Testing_set_begs.csvDataset(Testing) 

In [3]:
import numpy as np
import pandas as pd

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer


In [4]:
pharma_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Training_set_begs.csv')

In [5]:
pharma_data2 = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Training_set_begs.csv')

In [6]:
pharma_data

,ID_Patient_Care_Situation,Diagnosed_Condition,Patient_ID,Treated_with_drugs,Patient_Age,Patient_Body_Mass_Index,Patient_Smoker,Patient_Rural_Urban,Patient_mental_condition,A,B,C,D,E,F,Z,Number_of_prev_cond,Survived_1_year
0,22374,8,3333,DX6,56,18.479385,YES,URBAN,Stable,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0
1,18164,5,5740,DX2,36,22.945566,YES,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,6283,23,10446,DX6,48,27.510027,YES,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,5339,51,12011,DX1,5,19.130976,NO,URBAN,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,33012,0,12513,NaN,128,1.348400,Cannot say,RURAL,Stable,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23092,15613,3,1548,DX2 DX4,14,18.643448,NO,RURAL,Stable,1.0,1.0,0.0,1.0,1.0,1.0,0.0,5.0,0
23093,15503,41,2769,DX6,55,23.684585,NO,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
23094,2167,24,7671,DX6,63,27.500039,YES,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
23095,31646,11,5764,DX3,25,23.719125,YES,RURAL,Stable,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1


In [6]:
pharma_data.isnull().sum()

ID_Patient_Care_Situation       0
Diagnosed_Condition             0
Patient_ID                      0
Treated_with_drugs             13
Patient_Age                     0
Patient_Body_Mass_Index         0
Patient_Smoker                  0
Patient_Rural_Urban             0
Patient_mental_condition        0
A                            1235
B                            1235
C                            1235
D                            1235
E                            1235
F                            1235
Z                            1235
Number_of_prev_cond          1235
Survived_1_year                 0
dtype: int64

In [7]:
pharma_data['Treated_with_drugs'].fillna(pharma_data['Treated_with_drugs'].mode()[0], inplace=True)

In [8]:
pharma_data.Patient_Smoker.value_counts()

NO            13246
YES            9838
Cannot say       13
Name: Patient_Smoker, dtype: int64

In [9]:
pharma_data['Patient_Smoker'] = pharma_data['Patient_Smoker'].replace('Cannot say', 'NO')

In [10]:
pharma_data.Patient_Smoker.value_counts()

NO     13259
YES     9838
Name: Patient_Smoker, dtype: int64

In [11]:
num_cols = pharma_data.select_dtypes(include = np.number).columns    # getting all the categorical columns



In [12]:
pharma_data[num_cols] = pharma_data[num_cols].fillna(pharma_data[num_cols].median())

In [13]:
cat_cols = pharma_data.select_dtypes(include = 'object').columns    # getting all the categorical columns
cat_cols

Index(['Treated_with_drugs', 'Patient_Smoker', 'Patient_Rural_Urban',
       'Patient_mental_condition'],
      dtype='object')

In [14]:
pharma_data = pd.get_dummies(pharma_data, columns=cat_cols) 

In [15]:
pharma_data.head()

,ID_Patient_Care_Situation,Diagnosed_Condition,Patient_ID,Patient_Age,Patient_Body_Mass_Index,A,B,C,D,E,...,Treated_with_drugs_DX3 DX5,Treated_with_drugs_DX4,Treated_with_drugs_DX4 DX5,Treated_with_drugs_DX5,Treated_with_drugs_DX6,Patient_Smoker_NO,Patient_Smoker_YES,Patient_Rural_Urban_RURAL,Patient_Rural_Urban_URBAN,Patient_mental_condition_Stable
0,22374,8,3333,56,18.479385,1.0,0.0,0.0,0.0,1.0,...,0,0,0,0,1,0,1,0,1,1
1,18164,5,5740,36,22.945566,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,1,0,1
2,6283,23,10446,48,27.510027,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,1,1,0,1
3,5339,51,12011,5,19.130976,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,1,1
4,33012,0,12513,128,1.348400,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,1,0,1,0,1


In [16]:
X = pharma_data.drop('Survived_1_year', axis = 1) 


y = pharma_data['Survived_1_year']

In [60]:
np.random.seed(9)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 52)

In [71]:
RFC = RandomForestClassifier(min_samples_leaf=2, random_state=9)
RFC.fit(X_train,y_train)

RandomForestClassifier(min_samples_leaf=2, random_state=9)

In [72]:
pred = RFC.predict(X_test)

In [73]:
accuracy = accuracy_score(y_test, pred)
auc_roc1 = roc_auc_score(y_test, pred)
print("Accuracy score of the model with min sample leaf 2 is: {}".format(accuracy))
print("roc_auc_score of the model is: {}".format(auc_roc1))

Accuracy score of the model with min sample leaf 2 is: 0.8136363636363636
roc_auc_score of the model is: 0.7888828304467903


In [80]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.8136363636363636

### accuracy score using RandomForestClassifier is  0.8101

In [82]:
test_new = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Testing_set_begs.csv')

In [83]:
cat_cols = test_new.select_dtypes(include = 'object').columns    # getting all the categorical columns
cat_cols

Index(['Treated_with_drugs', 'Patient_Smoker', 'Patient_Rural_Urban',
       'Patient_mental_condition'],
      dtype='object')

In [84]:
test_new = pd.get_dummies(test_new, columns=cat_cols) 

In [85]:
Survived_1_year = RFC.predict(test_new)

In [89]:
Survived_1_year

array([1, 1, 1, ..., 1, 0, 0], dtype=int64)

In [90]:
res = pd.DataFrame(Survived_1_year)
res.index = test_new.index
res.columns = ["Survived_1_year"]
res.to_csv("result.csv")      